In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler # type: ignore
from tensorflow.keras.models import load_model # type: ignore

In [41]:
df = pd.read_csv('Data_In_2.csv')
df.rename(columns = {'company_name' : 'Company'}, inplace = True)

# Filter rows where 'Type' is equal to 'Health'
factor = 'Company'
search_term = 'AMAZON'
filtered_df = df[df[factor] == search_term]

In [42]:
data = filtered_df['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

In [43]:
def create_sequences(data, seq_length):
    sequences, labels = [], []
    
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length, 0]
        label = data[i + seq_length, 0]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [44]:
seq_length = 5
n_samples = len(data_scaled)
sequences, labels = create_sequences(data_scaled, seq_length)
split = int(0.9 * len(sequences))
X_test, y_test = sequences[split:], labels[split:]
X_train, y_train = sequences[:split], labels[:split]

In [45]:
predictor = load_model('model_lstm_8.keras')
amazn = predictor.predict(X_test)

22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step


In [46]:
amazn = scaler.inverse_transform(amazn)
amazn_y = scaler.inverse_transform(y_test.reshape(-1, 1))

In [47]:
dates_test = filtered_df['Date'].iloc[split + seq_length:]
df_plot_lstm = pd.DataFrame({'Date': dates_test[:len(amazn)], 'Real': amazn_y.flatten(), 'Predict': amazn.flatten()})

df_plot_lstm.tail(10)
# df_plot_lstm.to_csv('AMAZON_2.csv')

,Date,Real,Predict
32326,2024-05-03,186.210007,175.382278
32327,2024-05-06,188.699997,177.386703
32328,2024-05-07,188.759995,178.722702
32329,2024-05-08,188.000000,181.345108
32330,2024-05-09,189.500000,182.677353
32331,2024-05-10,187.479996,183.471542
32332,2024-05-13,186.570007,183.790024
32333,2024-05-14,187.070007,182.936172
32334,2024-05-15,185.990005,182.638489
32335,2024-05-16,183.600006,182.376343
